In [ ]:
import os
import sys
import openai
from dotenv import load_dotenv, find_dotenv

#  convert root dir path to absolute form
# append root path to sys.path to allow python to locate and import modules/packages from the root dir
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# read local .env file
_ = load_dotenv(find_dotenv(()))
openai.api_key = os.environ['OPENAI_API_KEY']

In [1]:
# Load documents
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="../documents/events.csv")
data = loader.load()
line = data[0]
line.metadata



{'source': '../documents/events.csv', 'row': 0}